DeFi Protocol Safety Scoring Tool

A lightweight Python-based risk assessment framework for decentralized finance, as of February 2026. Designed to run seamlessly in Google Colab, no heavy tools like Slither/Mythril



1. Introduction

The DeFi space is exciting but tricky—it's full of opportunities, but also packed with risks like wild price swings, liquidity drops and past hacks that still haunt some projects. This tool is my take on making sense of that chaos for a few big players: Aave, Uniswap and Compound.

Basically, it pulls together market stats and historical security info to give each protocol a quick safety score. The financial side looks at things like TVL scale and recent trends, while the security side digs into known exploits and how fresh their audits are. The end result is a clean PDF report with scores, charts and notes on where the data came from.

I kept it simple on purpose with no fancy blockchain security analysis tools like Slither, Mythril or endless dependencies, so anyone can run it in Colab seamlessly. It's not meant to be investment advice, just a practical way to spot relative risks. Think of it as a starting point for deeper dives.

Cell 1 – Setup & Constants

In [ ]:
!pip install scikit-learn fpdf2 beautifulsoup4 --quiet

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import re
from sklearn.linear_model import LinearRegression
from fpdf import FPDF
from fpdf.enums import XPos, YPos
from io import BytesIO
from google.colab import files
from datetime import datetime
from bs4 import BeautifulSoup

print("Setup complete.")

PROTOCOLS = {
    "aave": {
        "coingecko_id": "aave",
        "defillama_slug": "aave",
        "github_repo": "aave/aave-v3-core",
        "immunefi_url": "https://immunefi.com/bug-bounty/aave/information",
        "type": "lending"
    },
    "uniswap": {
        "coingecko_id": "uniswap",
        "defillama_slug": "uniswap",
        "github_repo": "Uniswap/v4-core",
        "immunefi_url": "https://cantina.xyz/bounties/f9df94db-c7b1-434b-bb06-d1360abdd1be",
        "type": "dex"
    },
    "compound": {
        "coingecko_id": "compound-governance-token",
        "defillama_slug": "compound-finance",
        "github_repo": "compound-finance/compound-protocol",
        "immunefi_url": "https://immunefi.com/bug-bounty/compoundfinance/information",
        "type": "lending"
    }
}

print(f"Configured {len(PROTOCOLS)} protocols: {', '.join(PROTOCOLS.keys())}")

2. Methodology

To build this, I focused on reliable, free data sources and straightforward calculations. No overcomplicating things with blockchain scanners or machine learning, just using APIs and basic math to keep it fast and explainable.


Data Collection (APIs & Public Sources)

Everything starts with grabbing fresh (or cached) data:
Market metrics: From CoinGecko's free API. This gives price changes over 7 and 30 days, plus a simple volatility measure (absolute 30-day % shift). It's quick and covers token basics without needing keys.

TVL and trends: Pulled from DefiLlama's protocol endpoint. Gets the current total locked value across chains and the last 30 daily snapshots for spotting declines. If the API hiccups (which happens in Colab), it falls back to recent cached values from early 2026 and notes that in the report.

For security, it's all from public reports, with no live scanning. I hardcoded summaries based on checks from sites like Rekt.news for exploits, protocol docs for audits and Immunefi for bounties. Easy to update if needed.

One thing to note: APIs can be flaky, so the code will flag when it uses backups. That way, you know if scores are live or not.

Cell 2 – Data Fetching Functions

In [ ]:
def fetch_market_data(protocol):
    config = PROTOCOLS[protocol]
    url = f"https://api.coingecko.com/api/v3/coins/{config['coingecko_id']}"
    headers = {'User-Agent': 'Mozilla/5.0'}                       # This reduces API blocks
    try:
        response_data = requests.get(url, headers=headers).json() # API request and parse JSON
        if 'market_data' not in response_data:
            raise KeyError('market_data')                         # to ensure the expected key exists
        market_data_response = response_data["market_data"]
        calculated_volatility = abs(market_data_response.get("price_change_percentage_30d_in_currency", {}).get("usd", 0)) / 100
        data = {
            "price_change_7d": market_data_response["price_change_percentage_7d_in_currency"]["usd"],
            "price_change_30d": market_data_response["price_change_percentage_30d_in_currency"]["usd"],
            "volatility": calculated_volatility
        }
        data['source'] = 'live'
        return data

    except Exception as e:
        print(f"Market data fetch failed for {protocol}: {e}")
        # 2026 updated fallbacks
        if protocol == "aave":
            data = {"price_change_7d": -15.0, "price_change_30d": 30.2, "volatility": 0.302}
        elif protocol == "uniswap":
            data = {"price_change_7d": -11.3, "price_change_30d": 35.2, "volatility": 0.352}
        else:
            data = {"price_change_7d": -20.6, "price_change_30d": 35.6, "volatility": 0.356}
        data['source'] = 'fallback'
        return data


def fetch_tvl_data(protocol):
    config = PROTOCOLS[protocol]
    url = f"https://api.llama.fi/protocol/{config['defillama_slug']}"
    headers = {'User-Agent': 'Mozilla/5.0'}
    try:
        defillama_response = requests.get(url, headers=headers).json()
        raw_tvl_history_data = defillama_response.get("tvl", [])
        tvl_history = [e["totalLiquidityUSD"] for e in raw_tvl_history_data][-30:]     # most recent 30 days
        chain_tvls = defillama_response.get("currentChainTvls", {})
        total_tvl = sum(v for k, v in chain_tvls.items() if '-borrowed' not in k.lower() and '-debt' not in k.lower())            # To handle potential key changes
        borrowed = sum(v for k, v in chain_tvls.items() if '-borrowed' in k.lower() or '-debt' in k.lower())
        chain_count = len(set(k.split('-')[0] for k in chain_tvls if '-borrowed' not in k.lower() and '-debt' not in k.lower()))  # Count unique chains supporting non-borrowed TVL

        # Attempt to get 24h volume (the API doesn't have it but adding it for completion's sake)
        volume_24h = defillama_response.get("volume_24h")
        if volume_24h is None:
            if protocol == "uniswap":
                volume_24h = 1.37e9
            else:
                volume_24h = 0        # Doesn't apply to lending

        data = {
            "total_tvl": total_tvl,
            "tvl_history": tvl_history or [0] * 30,
            "borrowed": borrowed,
            "chain_count": chain_count,
            "volume_24h": volume_24h
        }
        data['source'] = 'live'
        return data
    except Exception as e:
        print(f"TVL fetch failed for {protocol}: {e}")
        # DefiLlama data
        if protocol == "aave":
            data = {"total_tvl": 28.0e9, "tvl_history": [10e9] * 30, "borrowed": 7e9, "chain_count": 10, "volume_24h": 0}
        elif protocol == "uniswap":
            data = {"total_tvl": 3.105e9, "tvl_history": [5e9] * 30, "borrowed": 0, "chain_count": 40, "volume_24h": 1.37e9}
        else:
            data = {"total_tvl": 1.485e9, "tvl_history": [2e9] * 30, "borrowed": 0.6e9, "chain_count": 9, "volume_24h": 0}
        data['source'] = 'fallback'
        return data


def fetch_utilization_and_chains(protocol, tvl_data):
    config = PROTOCOLS[protocol]
    try:
        total_tvl = tvl_data['total_tvl']
        borrowed = tvl_data['borrowed']
        chains = tvl_data['chain_count']
        source = tvl_data['source']

        if config['type'] == 'lending':
            util = (borrowed / (total_tvl + borrowed)) * 100 if (total_tvl + borrowed) > 0 else 0
            metric_name = "Utilization (Lending)"
        elif config['type'] == 'dex':
            volume_24h = tvl_data['volume_24h']
            util = (volume_24h / total_tvl) * 100 if total_tvl > 0 else 0  # Volume/TVL as efficiency proxy (e.g., 44% for Uniswap based on fallbacks)
            util = min(util, 100)                                          # Cap to avoid outliers
            metric_name = "Efficiency (DEX - Vol/TVL)"
        else:
            util = 0
            metric_name = "N/A"
    except:
        print(f"Util/chains calc failed for {protocol}")
        # Fallbacks
        if protocol == "aave":
            util = 20.0  # Based on fallback borrowed/TVL
        elif protocol == "uniswap":
            util = 44.1  # ~1.37B volume / 3.1B TVL
        else:
            util = 28.8  # Based on fallback
        chains = PROTOCOLS[protocol].get('chain_count_fallback', 10)
        source = 'fallback'

    return {'utilization': util, 'chain_count': chains, 'source': source, 'metric_name': metric_name}

3. Risk Analysis (Financial & Historical Security)

The core is two risk angles, blended into one score.

Financial risk weighs current market health:
- Bigger TVL means more stability (min(total_tvl / 1e9, 10), capped at 10 to avoid over-weighting giants like Aave).
- Price momentum averages recent changes, positive if things are up, negative if down
(average of 7d + 30d % change / 5 – divided by 5 to normalize large swings).
- Volatility penalty: 30d absolute change × 2 – multiplier balances bear markets without overpowering. TVL trend uses a simple linear fit on the last 30 days, the downward slope pulls it lower.
- Trend penalty, negative slope from linear regression / 5e8 – divisor tunes impact for typical TVL fluctuations.


For historical security, it's about track record:
- Starts high at 10, then subtracts for any exploits (3x heavier if recent).
- More penalties for old or few audits, or noted vulns (even fixed ones, ×1.5 for multiples).
- Bonuses for high audit count (/4 min 3) recency (2025+ preferred) andactive bug bountiy (+1).

It's not perfect, it relies on public info, so it misses unreported issues. But it's transparent and based on verifiable events.



Data Processing & Visualization

Once data's in, processing is basic: linear regression for trends (sklearn), heatmaps/tables/charts with seaborn/matplotlib. Then fpdf assembles it into a PDF—summaries first, visuals after.
I added source tracking (live vs fallback) to keep things honest and clear.

Cell 3 – Analysis Functions (security history, trend prediction, scoring)

In [ ]:
def fetch_bounty_and_code_age(protocol):
    config = PROTOCOLS[protocol]
    headers = {'User-Agent': 'Mozilla/5.0'}
    try:
        # Live code age from GitHub API: Fetches the date of the last commit in project activity.
        github_url = f"https://api.github.com/repos/{config['github_repo']}/commits?per_page=1"     # GitHub API to get last commit
        commit_data = requests.get(github_url, headers=headers).json()
        last_commit_date = commit_data[0]['commit']['author']['date'].split('T')[0]
        code_age_days = (datetime.now() - datetime.strptime(last_commit_date, "%Y-%m-%d")).days

        # Live bounty from Immunefi/Cantina page using BS4 for parsing dynamic page structures to extract the maximum bounty size to reflect active security investment.
        response = requests.get(config['immunefi_url'], headers=headers)
        soup = BeautifulSoup(response.text, 'html.parser')
        for tag in soup.find_all(["script", "style"]):
            tag.extract()          # Remove all irrelevant tags for cleaner text extraction

        # Iterative search for common headings, to find the rewards section heading that indicates bounty information, usually near specific headers.
        reward_heading = None
        for heading in soup.find_all(['h1', 'h2', 'h3', 'h4']):
            head_text = heading.text.lower()
            if "reward" in head_text and any(word in head_text for word in ["threat", "severity", "impact", "level", "tier"]):
                reward_heading = heading
                break

        if reward_heading:
            section_content = []
            current = reward_heading.next_element
            # It collects content until the next major heading is found, to isolate the bounty description.
            while current and not (hasattr(current, 'name') and current.name in ['h1', 'h2', 'h3', 'h4']):
                if isinstance(current, str) and current.strip():
                    section_content.append(current.strip())
                current = current.next_element
            section_text = ' '.join(section_content)    # Combines relevant text for bounty parsing
        else:
            section_text = soup.get_text(separator=' ')

        slice_text = section_text.lower()[:4000]

        keywords = ["maximum reward", "max:", "critical", "rewards by threat level", "smart contract critical"]       # Locating the approximate position of bounty figures using a set of keywords.
        pos = min((slice_text.find(k) for k in keywords if slice_text.find(k) > -1), default=-1)                      # finds first occurrence of any bounty keyword
        if pos > -1:
            slice_text = slice_text[pos:pos + 2000]

        # Regex designed to be flexible on variations like 'max:', 'critical', dollar signs, commas, spaces and decimal values, common in bounty listings.
        matches = re.findall(r'(?:max(?:imum)?|critical)\s*[:\-\+]?\s*[\$]?\s*(\d{1,3}(?:[,\s]\d{3})*(?:\.\d+)?)', slice_text, re.I)
        bounty_size = 0
        for m in matches:
            try:
                cleaned = m.replace(',', '').replace(' ', '')
                num = float(cleaned)
                if 'million' in slice_text.lower() and num < 1000000:
                    num *= 1000000
                if num > bounty_size and 500000 <= num <= 20000000:
                    bounty_size = int(num)
            except:
                pass
        if bounty_size < 900000:
            raise ValueError("Parsed bounty suspiciously low or zero")       # Check parsed bounty value logically
        return {'bounty_size': bounty_size, 'code_age_days': code_age_days, 'source': 'live'}

    except Exception as e:
        print(f"Bounty/code fetch failed for {protocol}: {e}")
        # If the live fetching fails uses fallback values to ensure the report can still be generated.
        if protocol == "aave":
            bounty_size = 1000000
            code_age_days = 30
        elif protocol == "uniswap":
            bounty_size = 15500000
            code_age_days = 15
        else:
            bounty_size = 1000000
            code_age_days = 1335
        return {'bounty_size': bounty_size, 'code_age_days': code_age_days, 'source': 'fallback'}


def check_security_history(protocol, bounty_code_data):
    # 2026 data, simple and hardcoded to avoid overcomplication. Avoids complex or unreliable APIs for historical security events and their importance estimation.
    history = {
        "aave": {
            "exploit_count": 0,       # No major core (tools: periphery minor 2025 $51K)
            "recent_vuln": 2,         # 2023 mitigated, 2025 periphery (no loss)
            "audit_count": 12,        # V3.6 2025 (5), V4 2026 (Sherlock no findings + final)
            "latest_audit_year": 2026,
            "issues": ["2023 vulnerability mitigated, no loss; 2025 periphery hack $51K not core"],
            "has_audits": True,
            "audit_note": "V3.6 2025 (5 reports); V4 2026 (Sherlock contest no findings, final audits)",
            "bounty_active": True     # Active per tools
        },
        "uniswap": {
            "exploit_count": 0,       # No core (tools: hook-related like Cork 2025 $11M but was not core)
            "recent_vuln": 2,         # Hook vulns noted in audits, SIR 2025 exploit using V3 pools
            "audit_count": 10,        # V4 2024 (9 audits + $2.35M contest no severe)
            "latest_audit_year": 2024,
            "issues": ["No core exploits; hook vulnerabilities noted in audits; SIR 2025 using V3"],
            "has_audits": True,
            "audit_note": "V4 2024 (5+ in contest); multiple firms, $15.5M bug bounty active",
            "bounty_active": True
        },
        "compound": {
            "exploit_count": 1,       # 2021 $147M (tools: no new core, forks like Onyx/Sonne exploited)
            "recent_vuln": 0,         # No recent core (tools: old audits)
            "audit_count": 11,        # Mostly pre-2023 (OpenZeppelin/Trail of Bits)
            "latest_audit_year": 2020,
            "issues": ["No new 2023-2026 core; 2021 exploit $147M mitigated; forks vulnerable"],
            "has_audits": True,
            "audit_note": "Pre-2023 (Trail of Bits, OpenZeppelin); no recent audits found, $1M bounty active",
            "bounty_active": True
        }
    }
    data = history[protocol]
    data.update(bounty_code_data)

    base = 10
    audit_bonus = min(data["audit_count"] / 3, 4)     # Spread based on count
    exploit_penalty = data["exploit_count"] * 3
    vuln_penalty = data["recent_vuln"] * 1.5          # Higher for multiples
    bounty_bonus = 0.5 if data['bounty_size'] > 1000000 else 0
    code_penalty = -1 if data['code_age_days'] > 365 else 0
    recency_penalty = 0
    if data["latest_audit_year"] < 2026:
        recency_penalty = 0.5
    if data["latest_audit_year"] < 2025:
        recency_penalty = 1.5
    if data["latest_audit_year"] < 2023:
        recency_penalty = 3
    security = base - exploit_penalty - vuln_penalty - recency_penalty + audit_bonus + bounty_bonus + code_penalty
    security = max(0, min(10, security))

    return {
        "exploit_count": data["exploit_count"],
        "issues": data["issues"],
        "has_audits": data["has_audits"],
        "audit_note": data["audit_note"],
        "security": round(security, 2),
        "bounty_size": data["bounty_size"],
        "code_age_days": data["code_age_days"]
    }


def predict_tvl_trend(tvl_history):
    if len(tvl_history) < 2:
        return {"direction": "Unknown", "slope": 0}
    x = np.arange(len(tvl_history)).reshape(-1, 1)
    y = np.array(tvl_history)
    model = LinearRegression().fit(x, y)
    slope = model.coef_[0]
    direction = "Declining" if slope < -1e6 else "Stable/Increasing"  # A % calc would be more precise but -1e6 keeps it simple and works for the 3 protocols here
    return {"direction": direction, "slope": slope}


def compute_safety_scores(protocol, market, tvl, sec_data, trend, util_chains):
    config = PROTOCOLS[protocol]
    if not market or not tvl:
        return {"financial": 5, "security": 5, "operational": 5, "total": 5}

    price_trend = (market.get("price_change_7d", 0) + market.get("price_change_30d", 0)) / 2
    vol_penalty = market.get("volatility", 0) * 2
    tvl_size_score = min(tvl["total_tvl"] / 1e9, 10)
    trend_penalty = -abs(trend["slope"]) / 5e8 if trend["slope"] < 0 else 0
    util_penalty = -1 if (config['type'] == 'lending' and util_chains['utilization'] > 80) or (config['type'] == 'dex' and util_chains['utilization'] < 10) else 0
    chain_penalty = -0.5 if util_chains['chain_count'] < 3 else 0
    financial = tvl_size_score + price_trend / 5 - vol_penalty + trend_penalty + util_penalty + chain_penalty
    financial = round(max(-10, min(financial, 10)), 2)          # Allowing negative scores provides better differentiation for very high-risk scenarios

    operational_score = compute_operational_risk(protocol)
    security = sec_data["security"]

    total = round((financial + security + operational_score) / 3, 2)

    return {"financial": financial, "security": security, "operational": operational_score, "total": total}


def compute_operational_risk(protocol):
    # The notebook aims to be lightweight and avoid heavy external dependencies, so some data like this is pre-filled or simulated.
    try:
        if protocol == "aave":
            active_users = 150000
            oracle_risk = 1
        elif protocol == "uniswap":
            active_users = 200000
            oracle_risk = 1
        else:
            active_users = 50000
            oracle_risk = 1
        user_penalty = -1 if active_users < 100000 else 0
        op = 10 + user_penalty - oracle_risk
        op = max(0, min(10, op))
        return round(op, 2)
    except:
        return 8.0

Cell 4 – Visualization & PDF generation

In [ ]:
def plot_tvl_forecast(protocol, tvl_history):
    if len(tvl_history) < 2:
        return None
    x_historical_days = np.arange(-len(tvl_history) + 1, 1)
    y_historical_tvl_values = np.array(tvl_history)
    model = LinearRegression().fit(x_historical_days.reshape(-1,1), y_historical_tvl_values)
    x_future_days = np.arange(1, 8)
    y_forecasted_tvl_values = model.predict(x_future_days.reshape(-1,1))

    fig = plt.figure(figsize=(9,4.5))
    plt.plot(x_historical_days, y_historical_tvl_values / 1e9, 'o-', label="Historical", color="blue")
    plt.plot(x_future_days, y_forecasted_tvl_values / 1e9, '--', label="7-day forecast", color="red")
    plt.title(f"{protocol.upper()} TVL Trend & Forecast ($B)")
    plt.xlabel("Days (0 = most recent)")
    plt.ylabel("TVL ($B)")
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    return fig

def export_pdf_report(results):
    pdf = FPDF()
    pdf.set_auto_page_break(auto=True, margin=15)

    # Page 1: Per-protocol details
    pdf.add_page()
    pdf.set_font("Helvetica", "B", 14)
    pdf.cell(0, 10, "DeFi Protocol Safety Report (Feb 2026)", new_x=XPos.LMARGIN, new_y=YPos.NEXT, align="C")
    pdf.set_font("Helvetica", "I", 8)
    pdf.multi_cell(0, 5, "Disclaimer: Basic risk tool using public data. Not financial advice or full audit.")
    pdf.ln(5)

    for proto, res in results.items():
        pdf.set_font("Helvetica", "B", 12)
        pdf.cell(0, 5, f"{proto.upper():<10} {res['total']:.2f}/10 -> {res['rating']}", new_x=XPos.LMARGIN, new_y=YPos.NEXT)
        pdf.set_font("Helvetica", size=10)
        pdf.cell(0, 5, f"Financial : {res['financial']:>6.2f}/10 | Security: {res['security']:>6.2f}/10", new_x=XPos.LMARGIN, new_y=YPos.NEXT)
        pdf.cell(0, 5, f"TVL       : $ {res['tvl']['total_tvl']/1e9:>5.1f}B | Trend: {res['trend']['direction']}", new_x=XPos.LMARGIN, new_y=YPos.NEXT)
        pdf.cell(0, 5, f"{res['util_chains']['metric_name']}: {res['util_chains']['utilization']:.1f}% | Chains: {res['util_chains']['chain_count']}", new_x=XPos.LMARGIN, new_y=YPos.NEXT)
        pdf.cell(0, 5, f"Data Source: {res['data_source']}", new_x=XPos.LMARGIN, new_y=YPos.NEXT)
        pdf.cell(0, 5, f"Audited   : {'Yes' if res['sec']['has_audits'] else 'No'} | Exploits: {res['sec']['exploit_count']}", new_x=XPos.LMARGIN, new_y=YPos.NEXT)
        audit_note = res['sec']['audit_note']
        bounty_size = res['sec']['bounty_size']
        code_age_days = res['sec']['code_age_days']
        if bounty_size >= 1_000_000:
            bounty_display = f"${bounty_size / 1_000_000:.1f}M"
        else:
            bounty_display = f"${bounty_size:,}"
        pdf.multi_cell(0, 5, f"Audit Note: {audit_note} | Bounty: {bounty_display} | Code Age: {code_age_days} days", new_x=XPos.LMARGIN, new_y=YPos.NEXT)
        issues_str = '; '.join(res['sec']['issues']) or "None major"
        pdf.multi_cell(0, 5, f"Issues    : {issues_str}", new_x=XPos.LMARGIN, new_y=YPos.NEXT)
        pdf.ln(3)

    # Summary table
    pdf.ln(15)
    pdf.set_font("Helvetica", "B", 14)
    pdf.cell(0, 10, "Overall Protocol Summary", new_x=XPos.LMARGIN, new_y=YPos.NEXT, align="C")
    pdf.ln(5)
    col_widths = [40, 20, 20, 20, 20, 30, 20, 20]
    pdf.set_font("Helvetica", "B", 10)
    headers = ['Protocol', 'Total Score', 'Financial', 'Security', 'TVL(B)', 'Trend', 'Audit', 'Exploits']
    for i, header in enumerate(headers):
        pdf.cell(col_widths[i], 7, header, border=1, align='C')   # centering headers with borders
    pdf.ln()
    pdf.set_font("Helvetica", size=9)
    aligns = ['L', 'R', 'R', 'R', 'R', 'L', 'C', 'R']             # Alignments in table cells
    for proto, res in results.items():
        row_data = [
            proto.upper(),
            f"{res['total']:.2f}/10.00",
            f"{res['financial']:.2f}",
            f"{res['security']:.2f}",
            f"${res['tvl']['total_tvl']/1e9:.1f}",
            res['trend']['direction'],
            'Yes' if res['sec']['has_audits'] else 'No',
            str(res['sec']['exploit_count'])
        ]
        for i, item in enumerate(row_data):
            pdf.cell(col_widths[i], 6, item, border=1, align=aligns[i])
        pdf.ln()


    # Page 2: Heatmap (center=0 to handle negatives better)
    pdf.add_page()
    pdf.set_font("Helvetica", "B", 14)
    pdf.cell(0, 10, "Safety Heatmap (Higher = Safer)", new_x=XPos.LMARGIN, new_y=YPos.NEXT, align="C")
    pdf.ln(5)

    data = pd.DataFrame({
        "Protocol": [p.upper() for p in results],
        "Financial": [results[p]["financial"] for p in results],
        "Security": [results[p]["security"] for p in results],
        "Total": [results[p]["total"] for p in results]
    }).set_index("Protocol")

    fig, ax = plt.subplots(figsize=(8,4))
    sns.heatmap(data.T, annot=True, cmap="RdYlGn", center=5, fmt=".1f", cbar_kws={'label': 'Score'})  # Center=5, yellow for better representation of risk on a 0-10 scale
    plt.title("Component Scores")
    buf = BytesIO()
    plt.savefig(buf, format="png", dpi=200, bbox_inches="tight")
    plt.close()
    buf.seek(0)
    pdf.image(buf, x=15, w=180)

    y_after_heatmap_image = pdf.get_y()     # Store current Y position after heatmap

    # Footer if a fallback is used
    if any('Fallback' in res['data_source'] for res in results.values()):
        pdf.set_font("Helvetica", "I", 8)
        footer_height = 10
        y_bottom_of_page = pdf.h - pdf.b_margin - footer_height
        pdf.set_y(y_bottom_of_page)
        pdf.cell(0, footer_height, "Note: Fallback data from Feb 2026 cache used for some metrics.", align="C")
        pdf.set_y(y_after_heatmap_image)


    # Pages 3–5: TVL Charts
    for proto, res in results.items():
        pdf.add_page()
        pdf.set_font("Helvetica", "B", 14)
        pdf.cell(0, 10, f"TVL Trend - {proto.upper()}", new_x=XPos.LMARGIN, new_y=YPos.NEXT, align="C")
        pdf.ln(5)
        fig = plot_tvl_forecast(proto, res["tvl"]["tvl_history"])
        if fig:
            buf = BytesIO()
            fig.savefig(buf, format="png", dpi=200, bbox_inches="tight")
            plt.close(fig)
            buf.seek(0)
            pdf.image(buf, x=15, w=180)
        else:
            pdf.cell(0, 10, "Not enough data for chart", align="C")

    filename = "DeFi_Safety_Report.pdf"
    pdf.output(filename)
    print(f"PDF saved: {filename}")
    files.download(filename)
    return filename

Cell 5 – Run everything cell loop

In [ ]:
results = {}

for proto in PROTOCOLS:
    print(f"\nAnalyzing {proto.upper()}")
    market = fetch_market_data(proto)
    tvl = fetch_tvl_data(proto)
    util_chains = fetch_utilization_and_chains(proto, tvl)
    bounty_code = fetch_bounty_and_code_age(proto)
    sec = check_security_history(proto, bounty_code)
    trend = predict_tvl_trend(tvl["tvl_history"])
    scores = compute_safety_scores(proto, market, tvl, sec, trend, util_chains)

    if scores["total"] > 7:
        rating = "LOW RISK"
    elif scores["total"] > 4:
        rating = "MEDIUM RISK"
    else:
        rating = "HIGH RISK"

    sources = [market.get('source', 'unknown'), tvl.get('source', 'unknown'), util_chains.get('source', 'unknown')]
    data_source = 'Live API' if all(s == 'live' for s in sources) else 'Fallback (cached data used)'

    results[proto] = {
        "total": scores["total"],
        "financial": scores["financial"],
        "security": scores["security"],
        "operational": scores["operational"],
        "rating": rating,
        "tvl": tvl,
        "trend": trend,
        "sec": sec,
        "data_source": data_source,
        "util_chains": util_chains
    }

export_pdf_report(results)

4. Past Results & Analysis

Running the tool produces a new PDF, resembling the screenshots in the screenshots folder in the repo, which are also called below.
Actual numbers shift with market data and also depend on live API data vs fallback.

Example-Run Results Showcase

In [ ]:
# Important!!!

# The following results are not the most recent run output of the code. This cell showcases the uploaded screenshots from the screenshots folder in the repo, produced on a previous date.
# The output from your most recent run of the code is the PDF file downloaded on your system.


from IPython.display import Image

# Display the report summary page 1
github_image_url_1 = 'https://raw.githubusercontent.com/GeorgeKGM2058/DeFi-Protocol-Safety-Scoring-Tool/refs/heads/main/screenshots/report-summary-page1.png'
print("Report Summary - Page 1:")
display(Image(url=github_image_url_1))

# Display the report heatmap page 2
github_image_url_2 = 'https://raw.githubusercontent.com/GeorgeKGM2058/DeFi-Protocol-Safety-Scoring-Tool/refs/heads/main/screenshots/report-heatmap-page2.png'
print("\nReport Heatmap - Page 2:")
display(Image(url=github_image_url_2))

# Display the report TVL Aave page 3
github_image_url_3 = 'https://raw.githubusercontent.com/GeorgeKGM2058/DeFi-Protocol-Safety-Scoring-Tool/refs/heads/main/screenshots/report-tvl-aave-page3.png'
print("\nReport TVL Aave - Page 3:")
display(Image(url=github_image_url_3))

From that specific run:

- Aave often leads with mid-6s total, solid TVL cushions financial hits and fresh 2026 audits keep security near-perfect.
- Uniswap hovers around high-2s. Smaller TVL means more exposure to volatility, but strong bounty program and 2024 audits help security.
- Compound trails in low-2s: The old 2021 hack and dated audits drag security down, plus bear trends hit financial hard.

The heatmap makes differences pop. Greens for Aave's strengths, reds for others' financial weaknesses. TVL charts show steady declines across the board in this 2026 bear phase, with forecasts extending the line for a quick "what if" view.
Overall, it highlights how even top protocols vary. Scale protects Aave, but smaller ones like Compound feel every market dip more.
Limitations: Historical focus means it misses emerging threats and linear trends oversimplify volatility.

5. Conclusion

Wrapping up, this tool strips DeFi risk assessment down to essentials: Grab data, score simply, report clearly. It's lightweight enough for quick checks, but expandable.
Building it showed me how a few APIs and basic Python can turn raw numbers into useful insights.

References


CoinGecko API docs (2026) – https://www.coingecko.com/en/api

DefiLlama protocol data – https://defillama.com/docs/api

Rekt.news exploit database – https://rekt.news

Immunefi bounty listings – https://immunefi.com

Aave developer docs & audits – https://docs.aave.com

Uniswap governance & security – https://uniswap.org

Compound finance resources – https://compound.finance/docs